In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# Customers who reported to be Defrauded or Hacked - get this list from the Ops team

In [2]:
dfhacked = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\SupportingFile\Database of internal & external fraud incidents v2.xlsx", sheet_name = 'Main')

c:\Users\DwaipayanChakroborti\Myenv\reporting\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
dfhacked.columns

Index(['Source', 'Ticket Number', 'Ticket Status', 'Report Date', 'Fraud Type',
       'Sub MO', 'Disputed Amount', 'OFI', 'RFI', 'Complainant Name',
       'Contact Information', 'Source Account Number', 'With Police Report?',
       'TSA/Account Number', 'PID', 'Account Name', 'Segment/KYC', 'Address1',
       'Address2', 'Address3', 'Mobile Number', 'Account Status',
       'Date Blocked', 'TDB Account Balance', 'With TD?', 'With Active Loans?',
       'Callout Result', 'Disposition', 'Actions'],
      dtype='object')

In [4]:
pd.set_option("Display.max_columns", None)
dfhacked.head()

,Source,Ticket Number,Ticket Status,Report Date,Fraud Type,Sub MO,Disputed Amount,OFI,RFI,Complainant Name,Contact Information,Source Account Number,With Police Report?,TSA/Account Number,PID,Account Name,Segment/KYC,Address1,Address2,Address3,Mobile Number,Account Status,Date Blocked,TDB Account Balance,With TD?,With Active Loans?,Callout Result,Disposition,Actions
0,JIRA,CC-1686423,Closed,2023-05-22,Internal Fraud,Advance Fee Scam,1500.0,Various,CIMB via DragonPay,KATHLEEN ROSE BAUTISTA ANDAL,NaN,Various,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,Valid,"Joint investigation of InfoSec, AF Ops and HR ..."
1,NaN,CC-1698906,Closed,2023-05-26,NaN,NaN,1500.0,NaN,NaN,JESSA NA NAZ,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,CC-1698935,Closed,2023-05-26,NaN,NaN,2000.0,NaN,NaN,ROLLEN CARILLO REGINO,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,CC-1698352,Closed,2023-05-26,NaN,NaN,1500.0,NaN,NaN,JAIME FELIX REY III GENERAO CRUZ,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,CC-1698914,Closed,2023-05-26,NaN,NaN,1500.0,NaN,NaN,MAYDEN CORPUZ MAGLACION,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dfhacked['Disposition'].value_counts()

Disposition
Not Valid    162
Valid          2
Name: count, dtype: int64

In [6]:
dfhacked = dfhacked[dfhacked['Disposition']=='Valid']
dfhacked.shape

(2, 29)

In [7]:
# Replace spaces with underscores in column names
dfhacked.columns = dfhacked.columns.str.replace(' ', '_')
dfhacked.columns = dfhacked.columns.str.replace('/', '_')
dfhacked.columns = dfhacked.columns.str.replace('?', '')

In [8]:
dfhacked.dtypes

Source                           object
Ticket_Number                    object
Ticket_Status                    object
Report_Date              datetime64[ns]
Fraud_Type                       object
Sub_MO                           object
Disputed_Amount                 float64
OFI                              object
RFI                              object
Complainant_Name                 object
Contact_Information              object
Source_Account_Number            object
With_Police_Report               object
TSA_Account_Number              float64
PID                             float64
Account_Name                     object
Segment_KYC                      object
Address1                         object
Address2                         object
Address3                         object
Mobile_Number                   float64
Account_Status                   object
Date_Blocked             datetime64[ns]
TDB_Account_Balance              object
With_TD                          object


In [9]:
dfhacked1 = dfhacked[['Source', 'Ticket_Number', 'Ticket_Status', 'Report_Date', 'Fraud_Type',
       'Sub_MO', 'Disputed_Amount', 'OFI', 'RFI', 'Complainant_Name',
       'Contact_Information', 'With_Police_Report',
       'TSA_Account_Number', 'PID', 'Account_Name', 'Segment_KYC', 'Address1',
       'Address2', 'Address3', 'Mobile_Number', 'Account_Status',
       'Date_Blocked', 'With_TD', 'With_Active_Loans',
       'Callout_Result', 'Disposition', 'Actions']].copy()

In [10]:
from google.cloud import bigquery

# Define the dataset and table references
dataset_id = 'dap_ds_poweruser_playground'
table_id = 'Defraudedandhacked_mu'

# Define the table schema as per your DataFrame columns
schema = [
    bigquery.SchemaField("Source", "STRING"),
    bigquery.SchemaField("Ticket_Number", "STRING"),
    bigquery.SchemaField("Ticket_Status", "STRING"),
    bigquery.SchemaField("Report_Date", "TIMESTAMP"),
    bigquery.SchemaField("Fraud_Type", "STRING"),
    bigquery.SchemaField("Sub_MO", "STRING"),
    bigquery.SchemaField("Disputed_Amount", "FLOAT"),
    bigquery.SchemaField("OFI", "STRING"),
    bigquery.SchemaField("RFI", "STRING"),
    bigquery.SchemaField("Complainant_Name", "STRING"),
    bigquery.SchemaField("Contact_Information", "STRING"),
    bigquery.SchemaField("With_Police_Report", "STRING"),
    bigquery.SchemaField("TSA_Account_Number", "FLOAT"),
    bigquery.SchemaField("PID", "FLOAT"),
    bigquery.SchemaField("Account_Name", "STRING"),
    bigquery.SchemaField("Segment_KYC", "STRING"),
    bigquery.SchemaField("Address1", "STRING"),
    bigquery.SchemaField("Address2", "STRING"),
    bigquery.SchemaField("Address3", "STRING"),
    bigquery.SchemaField("Mobile_Number", "FLOAT"),
    bigquery.SchemaField("Account_Status", "STRING"),
    bigquery.SchemaField("Date_Blocked", "TIMESTAMP"),
    bigquery.SchemaField("With_TD", "STRING"),
    bigquery.SchemaField("With_Active_Loans", "STRING"),
    bigquery.SchemaField("Callout_Result", "STRING"),
    bigquery.SchemaField("Disposition", "STRING"),
    bigquery.SchemaField("Actions", "STRING"),
]

# Create the dataset reference
dataset_ref = client.dataset(dataset_id)

# Define the table reference
table_ref = dataset_ref.table(table_id)

# Configure the job to overwrite the table if it already exists
job_config = bigquery.LoadJobConfig(schema=schema)
job_config.autodetect = False
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

# Convert datetime columns to timestamp
for col in dfhacked.select_dtypes(include=['datetime64']):
    dfhacked[col] = dfhacked[col].astype('int64') // 10**9

# Load the DataFrame into BigQuery
job = client.load_table_from_dataframe(dfhacked1, table_ref, job_config=job_config)

# Wait for the job to complete
job.result()

print(f"Table {table_id} created in dataset {dataset_id}.")

Table Defraudedandhacked_mu created in dataset dap_ds_poweruser_playground.


In [11]:
sq = """select * from dap_ds_poweruser_playground.Defraudedandhacked_mu;"""
checkdf = client.query(sq).to_dataframe()
checkdf.shape

(2, 27)

# Query for Aquisition Channel

In [12]:
sq = """WITH installs AS 
(
SELECT
Event_Time,
CASE 
    ### social media ###
    -- anything from social
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%social%' THEN 'Social'
    ### direct ###
    -- fwb / referral
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'af_app_invites' THEN 'FWB'
    WHEN (Media_Source = 'invalid_media_source_name' OR Media_Source IS NULL) AND LOWER(Campaign) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%moengage%' THEN 'FWB'
    -- apple search
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%apple search ads%' THEN 'AppleSearch'
    -- facebook / meta
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Facebook Ads%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%meta%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'facebook' THEN 'Ads_Meta'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) = 'restricted' THEN 'Ads_Meta'
    -- google
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%googleadwords%' THEN 'Ads_Google'
      -- tiktok
    WHEN COALESCE(Media_Source,Partner,Campaign) like '%bytedanceglobal%' THEN 'TikTok_Paid'
    ### affiliates ###
    -- pokkt source
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%pokkt%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Tonik_CPA%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%campaign_name%' THEN 'Aff_Pokkt'
    -- tyr ads
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) IN ('tyrads_int','tjzymob_int','tyrads','ta_tonik_aos_ph') THEN 'Aff_TyrAds'
    -- sales doubler
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%salesdoubler%' THEN 'Aff_SalesDoubler'
     
    -- imoney
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%imoney%' THEN 'Aff_iMoney'
    -- moneymax
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%moneymax%' THEN 'Aff_MoneyMax'
    -- jeff
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%jeff%' THEN 'Aff_Jeff'
    -- shareit
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%shareit%' THEN 'Aff_ShareIt'
     
    -- appnext
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%appnext%' THEN 'Aff_AppNext'
    -- mediadonuts / entravision
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%mediadonuts%' THEN 'Aff_Mediadonuts'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%entravision%' THEN 'Aff_Mediadonuts'
    -- shopback
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%shopback_int%' THEN 'Aff_Shopback'
    -- deepsea
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%deepsea%' THEN 'Aff_DeepSea'
    ## organic ##
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('af_banner') THEN 'af_Banner'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%sendgrid%' THEN 'Organic'
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('Website') THEN 'Website'
    
    ## owned media ##
    WHEN media_source IN ('PRFlexLoans_WebAndroidPressRelease') THEN 'Owned Media'
    
    WHEN media_source IN ('invalid_media_source_name') THEN 'Invalid Media Source'
    WHEN media_source IN ('None') THEN 'Broken OneLink'
ELSE "Organic"
END as Source,
AppsFlyer_ID,
Customer_User_ID,
Device_Type
    
FROM
(
    SELECT Install_Time,Event_Time,AppsFlyer_ID,Customer_User_ID,Platform,Partner,Media_Source,Campaign,Device_Type,OS_Version
    FROM `prj-prod-dataplatform.appsflyer_raw.organic_installs_report` I --Organic Installs
    WHERE Date(Event_Time) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 11 MONTH) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
    AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 350 DAY) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
    UNION ALL
    SELECT Install_Time,Event_Time,AppsFlyer_ID,Customer_User_ID,Platform,Partner,Media_Source,Campaign,Device_Type,OS_Version
    FROM `prj-prod-dataplatform.appsflyer_raw.installs_report` I -- Non-Organic Installs
    WHERE Date(Event_Time) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 11 MONTH) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
    AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 350 DAY) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
    
)
WHERE 1=1
AND event_time >= '2022-09-01'
)
, registrations AS 
(
SELECT
MAX(Event_Time) event_time,
CASE 
    ### social media ###
    -- anything from social
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%social%' THEN 'Social'
    ### direct ###
    -- fwb / referral
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'af_app_invites' THEN 'FWB'
    WHEN (Media_Source = 'invalid_media_source_name' OR Media_Source IS NULL) AND LOWER(Campaign) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%moengage%' THEN 'FWB'
    -- apple search
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%apple search ads%' THEN 'AppleSearch'
    -- facebook / meta
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Facebook Ads%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%meta%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'facebook' THEN 'Ads_Meta'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) = 'restricted' THEN 'Ads_Meta'
    -- google
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%googleadwords%' THEN 'Ads_Google'
      -- tiktok
    WHEN COALESCE(Media_Source,Partner,Campaign) like '%bytedanceglobal%' THEN 'TikTok_Paid'
    ### affiliates ###
    -- pokkt source
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%pokkt%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Tonik_CPA%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%campaign_name%' THEN 'Aff_Pokkt'
    -- tyr ads
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) IN ('tyrads_int','tjzymob_int','tyrads','ta_tonik_aos_ph') THEN 'Aff_TyrAds'
    -- sales doubler
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%salesdoubler%' THEN 'Aff_SalesDoubler'
     
    -- imoney
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%imoney%' THEN 'Aff_iMoney'
    -- moneymax
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%moneymax%' THEN 'Aff_MoneyMax'
    -- jeff
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%jeff%' THEN 'Aff_Jeff'
    -- shareit
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%shareit%' THEN 'Aff_ShareIt'
     
    -- appnext
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%appnext%' THEN 'Aff_AppNext'
    -- mediadonuts / entravision
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%mediadonuts%' THEN 'Aff_Mediadonuts'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%entravision%' THEN 'Aff_Mediadonuts'
    -- shopback
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%shopback_int%' THEN 'Aff_Shopback'
    -- deepsea
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%deepsea%' THEN 'Aff_DeepSea'
    ## organic ##
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('af_banner') THEN 'af_Banner'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%sendgrid%' THEN 'Organic'
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('Website') THEN 'Website'
    
    ## owned media ##
    WHEN media_source IN ('PRFlexLoans_WebAndroidPressRelease') THEN 'Owned Media'
    
    WHEN media_source IN ('invalid_media_source_name') THEN 'Invalid Media Source'
    WHEN media_source IN ('None') THEN 'Broken OneLink'
ELSE "Organic"
END as Source,
AppsFlyer_ID,
Device_Type,
-- Customer_User_ID
FROM
(

  SELECT MIN(Event_time) event_time,Media_source,Campaign,Partner,AppsFlyer_ID,Install_Time,Platform,OS_Version,Device_Type
  FROM `prj-prod-dataplatform.appsflyer_raw.in_app_events_report`  -- Non-Organic Registrations
  WHERE (Event_Name='Onboarding_Success' OR Event_Name LIKE 'Onboarding_Success_Dashboard%')
  AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 350 DAY) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
  AND DATE(Event_Time) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 11 MONTH) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
  GROUP BY 2,3,4,5,6,7,8,9
  UNION ALL
      
  SELECT MIN(Event_time) event_time,Media_source,Campaign,Partner,AppsFlyer_ID,Install_Time,Platform,OS_Version,Device_Type
  FROM `prj-prod-dataplatform.appsflyer_raw.organic_in_app_events_report`  -- Organic Registrations
  WHERE (Event_Name='Onboarding_Success' OR Event_Name LIKE 'Onboarding_Success_Dashboard%')
  AND DATE(_PARTITIONTIME) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 380 DAY) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
  AND DATE(Event_Time) BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 11 MONTH) AND DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY)
  GROUP BY 2,3,4,5,6,7,8,9
    
)
WHERE 1=1
AND event_time >= '2022-09-01'
GROUP BY 2,3,4
)
, applications AS 
(
SELECT 
decision_date,
IF(reloan_flag=1,'Re-loan',new_loan_type) loan_type,
CASE 
    ### social media ###
    -- anything from social
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%social%' THEN 'Social'
    ### direct ###
    -- fwb / referral
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'af_app_invites' THEN 'FWB'
    WHEN (Media_Source = 'invalid_media_source_name' OR Media_Source IS NULL) AND LOWER(Campaign) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%moengage%' THEN 'FWB'
    -- apple search
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%apple search ads%' THEN 'AppleSearch'
    -- facebook / meta
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Facebook Ads%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%meta%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'facebook' THEN 'Ads_Meta'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) = 'restricted' THEN 'Ads_Meta'
    -- google
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%googleadwords%' THEN 'Ads_Google'
      -- tiktok
    WHEN COALESCE(Media_Source,Partner,Campaign) like '%bytedanceglobal%' THEN 'TikTok_Paid'
    ### affiliates ###
    -- pokkt source
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%pokkt%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Tonik_CPA%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%campaign_name%' THEN 'Aff_Pokkt'
    -- tyr ads
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) IN ('tyrads_int','tjzymob_int','tyrads','ta_tonik_aos_ph') THEN 'Aff_TyrAds'
    -- sales doubler
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%salesdoubler%' THEN 'Aff_SalesDoubler'
     
    -- imoney
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%imoney%' THEN 'Aff_iMoney'
    -- moneymax
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%moneymax%' THEN 'Aff_MoneyMax'
    -- jeff
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%jeff%' THEN 'Aff_Jeff'
    -- shareit
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%shareit%' THEN 'Aff_ShareIt'
     
    -- appnext
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%appnext%' THEN 'Aff_AppNext'
    -- mediadonuts / entravision
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%mediadonuts%' THEN 'Aff_Mediadonuts'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%entravision%' THEN 'Aff_Mediadonuts'
    -- shopback
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%shopback_int%' THEN 'Aff_Shopback'
    -- deepsea
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%deepsea%' THEN 'Aff_DeepSea'
    ## organic ##
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('af_banner') THEN 'af_Banner'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%sendgrid%' THEN 'Organic'
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('Website') THEN 'Website'
    
    ## owned media ##
    WHEN media_source IN ('PRFlexLoans_WebAndroidPressRelease') THEN 'Owned Media'
    
    WHEN media_source IN ('invalid_media_source_name') THEN 'Invalid Media Source'
    WHEN media_source IN ('None') THEN 'Broken OneLink'
ELSE "Organic"
END as Source,
    
digitalLoanAccountId,
flagApproval,
customerId,
Appsflyer_id
FROM `prj-prod-dataplatform.risk_credit_mis.loan_master_table` LA
WHERE 1=1 
AND DATE(LA.decision_date) >= '2023-07-01'
AND new_loan_Type IN ('Quick','Flex')
ORDER BY 1 
)
select customerId, Appsflyer_id, digitalLoanAccountId,Source  
, case when Source like 'Organic' then 'Organic' else 'InOrganic' end flag_organic_inorganic
from applications;"""

In [13]:
dfaquisition = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfaquisition.shape

Job ID 00baa8f6-fed4-41af-82bc-0be31d153b57 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(456893, 5)

In [14]:
dfaquisition.columns

Index(['customerId', 'Appsflyer_id', 'digitalLoanAccountId', 'Source',
       'flag_organic_inorganic'],
      dtype='object')

# Query for the rest of the part

In [15]:
sq = """with 
devicemodel as 
(with base as
(select distinct lmt.digitalLoanAccountId,
  t4.GeneralInfo.brand     ,
  t4.Hardware.device__brand   ,
  t4.Hardware.device__manufacturer   ,
  t4.Hardware.device__model
from`risk_credit_mis.loan_master_table` lmt
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` t2
ON lmt.digitalLoanAccountId = t2.digitalLoanAccountId
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_credolab_track` t3
ON t2.credolabRefNumber = t3.refno
LEFT JOIN
`prj-prod-dataplatform.credolab_raw.android_credolab_datasets_struct_columns` t4
ON t3.refno = t4.deviceId
)
select digitalLoanAccountId, device__manufacturer devicemanufacturer, device__model devicemodel, device__brand devicebrand from base),
employmenttype as (select * from 
(select distinct digitalLoanAccountId, OccupationStatusDesc, row_number() over(partition by digitalLoanAccountId order by LastUpdateDate desc) rnk 
from prj-prod-dataplatform.risk_credit_cic_data.employment_data where OccupationStatusDesc is not null
)
where rnk = 1),
loanlocation as (select * from 
(select digitalLoanAccountId,
created_dt,
updated_dt,
latitude,
longitude,
row_number() over(partition by digitalLoanAccountid order by created_dt desc) rnk
 from prj-prod-dataplatform.dl_loans_db_raw.tdbk_customer_location
)
where rnk = 1),
base as 
(select * from
(select  distinct customerId,user_id,tsaOnboardingDateTime,credolab_ReferenceNumber,deviceType,
digitalLoanAccountId,case when loan_type in ('Flex-up','Reloan') then 'Repeat Applicant' 
when First_application=1 and customer_id_first_applicant is not null then 'New Applicant'
when accept_cust_id is null then  'Prev_Rejected_Reapply'
when rn=1 then 'New Applicant' 
--when Reject_application_count=1 then 'New Applicant' 
when accept_application_count>1 and accept_cust_id is not null then  'Repeat Applicant'

else 'Repeat Applicant' 
end as applicant_type,loan_type,loan_sub_type,sil_product,application_Submission_DateTime,
fraudScore,credolabScore,decision,applicationStatus,disbursementFlag,flg_mature_fpd10,flg_mature_fpd30,
flg_mature_fspd_30,flg_mature_fstpd_30,deffpd10,deffpd30,deffspd30,deffstpd30

from (with deliquency as
(select loanAccountNumber,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30  in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),

accept_application_cnt as (select customerId,count(*) as accept_application_count
from risk_credit_mis.loan_master_table loan where loan.new_loan_type in ('Flex-up','SIL-Instore','SIL-Online(discontinued)','Quick','SIL ZERO','ACL Patner 711')
  and (flagApproval = 1 or flagRejection = 1 or loan.new_loan_type in ('Flex-up')) and coalesce(loan.termsAndConditionsSubmitDateTime,if (loan.new_loan_type ='Flex-up',loan.startApplyDateTime,loan.termsAndConditionsSubmitDateTime))  is not null and applicationStatus in ('ACTIVATED','COMPLETED','APPROVED',
'INPROGRESS','SIGNEDDOCCOMPLETED','ACCEPT') group by 1 order by customerId),


applciant_type_first as (select customerId customer_id_first_applicant,count(*) First_application from `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
where new_loan_type in ('Flex-up','SIL-Instore','SIL-Online(discontinued)','Quick','SIL ZERO','ACL Patner 711')
  and (flagApproval = 1 or flagRejection = 1 or new_loan_type in ('Flex-up')) and coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime))  is not null group by 1 having count(*)=1),

previous_loan_product as (select loan.customerId,
    case when loan.new_loan_type ='Flex-up' and reloan_flag=1 then 'Reloan'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
     WHEN loan.new_loan_type ='Flex' then 'Flex'
    when loan.new_loan_type ='Quick'  then 'Quick'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
    when loan.new_loan_type in ('SIL-Instore','SIL-Online(discontinued)','SIL ZERO') then 'SIL'
    when loan.new_loan_type in ('ACL Patner 711') then '711'
    end as prev_new_loan_type,loan.loanType prev_loan_type,prev_loan.previous_loan_acct,prev_loan.loanAccountNumber,loan.startApplyDateTime from `risk_credit_mis.loan_master_table` loan
join
(select * from (select customerId, digitalLoanAccountId,loanAccountNumber,new_loan_type,startApplyDateTime, lead(loanAccountNumber)
 OVER(PARTITION BY customerId order by startApplyDateTime desc) as previous_loan_acct from `risk_credit_mis.loan_master_table`
 where applicationStatus in ('COMPLETED','ACTIVATED')  order by customerId) where --previous_loan_acct is not null and 
 new_loan_type='Flex-up')prev_loan on loan.loanAccountNumber=prev_loan.previous_loan_acct)
  
  select
  loan.customerId,
  cust.user_id,
  tsa_onboarding_time as tsaOnboardingDateTime,
  credolabRefNumber as credolab_ReferenceNumber,
  deviceType,
  loan.digitalLoanAccountId,
  --case when applciant_type_first.disbursed_loan >1 then 'Repeat Applicant' else 'New Applicant' end as applicant_type,

    case when loan.new_loan_type ='Flex-up' and reloan_flag=1 then 'Reloan'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
    when loan.new_loan_type ='Quick'  then 'Quick'
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then 'Flex-up'
    when loan.new_loan_type in ('SIL-Instore','SIL-Online(discontinued)','SIL ZERO') then 'SIL'
    when loan.new_loan_type in ('ACL Patner 711') then '711'
    end as loan_type,
    case when loan.new_loan_type ='Flex-up' and reloan_flag=1 then coalesce('Reloan_'||prev_new_loan_type)
    when loan.new_loan_type ='Flex-up' and reloan_flag=0 then coalesce('Flex-up_'||prev_new_loan_type)
    else loan.new_loan_type end loan_sub_type,
    
    --new_loan_type as sub_loan_type,reloan_flag,
    --prev_new_loan_type, 
    case when loan.loantype='BNPL' and store_type =1 then 'Appliance'
    when loan.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loan.loantype='BNPL' and store_type =3 then 'Mall' 
    when loan.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as sil_product,
    coalesce(loan.termsAndConditionsSubmitDateTime,if (loan.new_loan_type ='Flex-up',loan.startApplyDateTime,loan.termsAndConditionsSubmitDateTime)) application_Submission_DateTime,
    fraudScore,credolabScore,
      case
      when flagApproval = 1 then 'approved'
      when flagRejection = 1 then 'rejected'
      else 'auto-approved without model'
    end as decision,
    applicationStatus,
    flagDisbursement as disbursementFlag,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 40, 1 ,0) as flg_mature_fpd10,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 60, 1 ,0) as flg_mature_fpd30,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 90, 1 ,0) as flg_mature_fspd_30,
    IF(Date_diff(current_date(),date(disbursementDateTime), DAY) >= 120, 1 ,0) as flg_mature_fstpd_30,
    deliquency.deffpd10,deliquency.deffpd30,deliquency.deffspd30,deliquency.deffstpd30,row_number() over (partition by loan.customerId order by coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',loan.startApplyDateTime,termsAndConditionsSubmitDateTime))) rn,accept_application_count,accept_application_cnt.customerId accept_cust_id,First_application,customer_id_first_applicant

   
  from prj-prod-dataplatform.risk_credit_mis.loan_master_table loan
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name) sil_category on purpleKey=sil_category.mer_refferal_code
  left join deliquency on deliquency.loanAccountNumber=loan.loanAccountNumber
  left join previous_loan_product on previous_loan_product.loanAccountNumber=loan.loanAccountNumber
  left join applciant_type_first on applciant_type_first.customer_id_first_applicant=loan.customerId
  left join dl_customers_db_raw.tdbk_customer_mtb cust on cust.cust_id=cast(loan.customerId as string)
  left join accept_application_cnt on accept_application_cnt.customerId=loan.customerId
  where loan.new_loan_type in ('Flex-up','SIL-Instore','SIL-Online(discontinued)','Quick','SIL ZERO','ACL Patner 711')
  and (flagApproval = 1 or flagRejection = 1 or loan.new_loan_type in ('Flex-up')) and coalesce(loan.termsAndConditionsSubmitDateTime,if (loan.new_loan_type ='Flex-up',loan.startApplyDateTime,loan.termsAndConditionsSubmitDateTime))  is not null) final where loan_sub_type is not null)
), 
delqtab as 
(select loanAccountNumber,  
case when obs_min_inst_def30 >=2 then 1 else 0 end obsFSPD30,
case when min_inst_def30 in (1,2) then 1 else 0 end fspd30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
Excl_evertransacted as 
(select distinct cast(customer_id as numeric) customerId from prj-prod-dataplatform.risk_mart.customer_transactions where status like 'Success'),
Excl_qualified_FSPD30 as
(select loanAccountNumber, obs_min_inst_def30 from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data where obs_min_inst_def30 >= 2),
Excl_blacklisted_clean as 
(select customerid, matching_attribute, id, refresh_date from  prj-prod-dataplatform.CRM_RISK.fraud_blacklist_clean),
educate as 
    (select distinct edu.digitalLoanAccountId, edu.education_id, edu1.description
    from `prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_purpose` edu
    inner join (select id, description from dl_loans_db_raw.tdbk_loan_lov_mtb where module = 'Education') edu1 on edu.education_id = edu1.id
    ),
    educate2 as 
    (select *, row_number() over(partition by digitalLoanAccountId order by education_id desc) rnk from educate),
    educate3 as 
    (select * from educate2 where rnk = 1),
    education as 
    (
    select 
    lmt.customerId,
    educate3.digitalLoanAccountId,
    educate3.education_id,
    educate3.description Education_Type, 
    educate3.rnk educationrnk
    from educate3 
    inner join prj-prod-dataplatform.risk_credit_mis.loan_master_table lmt on lmt.digitalLoanAccountId = educate3.digitalLoanAccountId
    )
    ,
    digitalloanapplication as (select 
    digitalLoanAccountId, 
    monthlyInterestRate , 
    interest_rate_offer ,
    monthlyIncomeCheck, 
    from prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application ), 
    loantypeselection as 
    (SELECT 
    digitalLoanAccountId 
    FROM `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
    where date(disbursementDateTime) >= '2023-07-01'
    and flagDisbursement = 1
    and kycStatus = 'SKYC'
    -- and customerId in (select customerId from Excl_evertransacted) ---Excluded customers who never transacted
    -- and loanAccountNumber in (select loanAccountNumber from Excl_qualified_FSPD30) --- Exclude customer who has not yet matured FSPD30
    -- and customerId not in (select distinct customerid from Excl_blacklisted_clean) --- Exclude customer who are blacklisted
    -- and customerId not in (select distinct customerId from `risk_credit_mis.loan_master_table` where loanPaidStatus in ('Written Off', 'Sold')) --- Exclude written off
    -- and customerId not in (Select PID from dap_ds_poweruser_playground.Defraudedandhacked_mu where PID is not null)
    ),
    Appreinstall as 
    (select cast(customer_id as numeric) customerId, count(customer_id)  from moengage_raw.events_hourly where event_code = 'REINSTALL'
        group by 1
        having count(customer_id) > 1)
    ,
    loan
    as
    (select 
    distinct
    lmt.digitalLoanAccountId ,
    lmt.customerId ,
    lmt.loanAccountNumber ,
    lmt.tsa_onboarding_time,
    lmt.startApplyDateTime,
    lmt.decision_date,
    lmt.termsAndConditionsSubmitDateTime,
    lmt.disbursementDateTime, 
    lmt.new_loan_type,
    lmt.gender,
    lmt.age,
    education.Education_Type,
    education.education_id,
    et.OccupationStatusDesc Employment_type,
    lmt.custLoanJourneyCity,
    lmt.custLoanJourneyProvince,
    lmt.custLoanJourneyZipCode,
    ll.latitude loanapplication_latitude,
    ll.longitude loanapplication_longitude,
    lmt.province,
    lmt.city,
    lmt.postalCode,
    lmt.geolocation onboardinggeolocation,
    lmt.incomeperSalaryScale,
    lmt.monthlyIncome,
    lmt.maritalStatus,
    lmt.purposeDescription,
    ascv.mer_category,
    lmt.isfatca,
    lmt.docType,
    lmt.loanRequestAmount,
    lmt.loanRequestTenure,
    dlp.monthlyInterestRate,
    dlp.interest_rate_offer,
    dlp.monthlyIncomeCheck,
    lmt.kycStatus,
	case when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%ios%' then 'ios'
	     when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%andro%' then 'android'
		 else 'other' end Device_os_type,
    delqtab.obsFSPD30,
    delqtab.FSPD30,
	  case when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%ios%' then coalesce(cast(SUBSTRING(coalesce(lmt.osVersion, lmt.osversion_v2), 4, 2)  as numeric),0)
	     when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%andro%' then coalesce(cast(SUBSTRING(coalesce(lmt.osVersion, lmt.osversion_v2), 8, 2)  as numeric),0)
		 end Device_OSVersion,
   lmt.deviceType,
    b.Network Network_provider,
    lmt.brand loanmaster_brand,
    dm.devicemanufacturer,
    dm.devicemodel,
    dm.devicebrand brand, 
	case when eet.customerId is null then 1 else 0 end customer_never_transacted_flag,
	case when eqf.loanAccountNumber is not null then 1 else 0 end fspd30_mature_flag,
	case when ebc.customerId is not null then 1 else 0 end blacklisted_customer_flag,
	case when lmt.customerId in (select distinct customerId from `risk_credit_mis.loan_master_table` where loanPaidStatus in ('Written Off', 'Sold')) then 1 else 0 end writtenoff_flag,
	case when lmt.customerId in (Select PID from dap_ds_poweruser_playground.Defraudedandhacked_mu where PID is not null) then 1 else 0 end hackedordefrauded_flag,
  case when e.vas_flag = 'true' then 1 else 0 end as VasFlag,
  case when Appreinstall.customerId is not null then 1 else 0 end EverReinstallFlag,
  lmt.credolabScore
    from base 
    left join   `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = base.digitalLoanAccountId
    inner join loantypeselection lts on lts.digitalLoanAccountId = lmt.digitalLoanAccountId 
    inner join delqtab on delqtab.loanAccountNumber = lmt.loanAccountNumber
    Left join education on education.digitalLoanAccountId = lmt.digitalLoanAccountId
    left join digitalloanapplication dlp on dlp.digitalLoanAccountId = lmt.digitalLoanAccountId
    LEFT JOIN (select *, row_number() over(partition by Number_Prefix order by Network) rnk 
  from `prj-prod-dataplatform.manual_source_extracts.mobile_carrier_mapping`) b ON LEFT(RIGHT(lmt.mobileNo,LENGTH(lmt.mobileNo)-2),3) = CAST(b.number_prefix AS STRING)
	 left join Excl_evertransacted eet on eet.customerId = lmt.customerId
	 left join Excl_qualified_FSPD30 eqf on eqf.loanAccountNumber = lmt.loanAccountNumber
	 left join Excl_blacklisted_clean ebc on ebc.customerId = lmt.customerId
   left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application e
          on e.loanAccountNumber = lmt.loanAccountNumber
   left join Appreinstall on Appreinstall.customerId = lmt.customerId
   left join prj-prod-dataplatform.worktable_datachampions.AREA_SPLIT_CURRENTV2 ascv on ascv.purpleKey = lmt.purpleKey
   left join loanlocation ll on ll.digitalLoanAccountId = lmt.digitalLoanAccountId
   left join employmenttype et on et.digitalLoanAccountId = lmt.digitalLoanAccountId
   left join devicemodel dm on dm.digitalLoanAccountId = lmt.digitalLoanAccountId
    where b.rnk = 1
    )
    select * from loan ;"""

In [16]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID a8da799a-97ee-4944-82a1-c804ffb50267 successfully executed: |          | 
Downloading: 100%|██████████|


In [17]:
df.shape 

(135236, 54)

# Events Before Apply

In [18]:
sq = """with eventbase as 
(select Customer_User_ID,Event_Time, Event_Name  from prj-prod-dataplatform.appsflyer_raw.in_app_events_report where Customer_User_ID is not null 
union all
select Customer_User_ID,Event_Time, Event_Name from prj-prod-dataplatform.appsflyer_raw.organic_in_app_events_report where Customer_User_ID is not null 
),
eventsbeforeapply as 
(select a.customerId, a.digitalLoanAccountId , a.startApplyDateTime 
, eb.Customer_User_ID, eb.Event_Time, eb.Event_Name
from `risk_credit_mis.loan_master_table` a
left join eventbase eb on eb.Customer_User_ID  = cast(a.customerId as string) and date(a.startApplyDateTime) > date(eb.Event_Time)
),
eventsbeforeapplybase as
(select digitalLoanAccountId, startApplyDateTime, count(distinct Event_Name) cntevents 
from eventsbeforeapply
group by 1,2
)
select * from eventsbeforeapplybase;"""

eventbfrapplydf = client.query(sq).to_dataframe(progress_bar_type='tqdm')
eventbfrapplydf

Job ID 9865aa9b-477c-4bc1-9ce4-513e539ce8ea successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,digitalLoanAccountId,startApplyDateTime,cntevents
0,10f58b4e-81cf-4a65-9608-520041f1dcd1,2023-04-24 11:39:01,25
1,5c0e0297-8977-4514-805e-e1280a78603b,2024-09-19 06:16:22,98
2,811f527f-dca7-4846-88a0-6281bb5a13a6,2024-05-27 10:27:26,37
3,45578d4c-7e32-4e46-9d56-a4e9e548d2bc,2023-08-23 12:35:25,85
4,9d0c30e0-0b0f-45ec-888f-af0095cfc857,2023-11-08 20:39:18,102
...,...,...,...
1822543,88bd6a5c-01ff-4b32-a3aa-156c0962e3a2,2023-09-09 11:15:53,0
1822544,5fea3d5e-5924-47ac-966c-b5778cf2365b,2024-10-13 15:13:31,0
1822545,bb578380-ae6a-453f-8053-0c1899b72518,NaT,0
1822546,a3af50d7-65cd-4d7f-a898-0fc0f6d70c4d,2024-07-09 11:56:29,0


# Saving the dataframes to csv files

In [19]:
dfhacked1.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\hackedconfirmedbyoperationsteam.csv", index = False)
dfaquisition.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\aquisitiondatareceivedfromIvanah.csv", index = False)
df.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\otherpart.csv", index = False)
eventbfrapplydf.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\eventsbeforeapplydate.csv", index = False)

In [20]:
finaldf = df.merge(dfaquisition, on='digitalLoanAccountId', how = 'left')
finaldf = finaldf.merge(eventbfrapplydf[['digitalLoanAccountId', 'cntevents']], on='digitalLoanAccountId', how = 'left')
finaldf.shape

(135236, 59)

In [21]:
finaldf.sample(5)

,digitalLoanAccountId,customerId_x,loanAccountNumber,tsa_onboarding_time,startApplyDateTime,decision_date,termsAndConditionsSubmitDateTime,disbursementDateTime,new_loan_type,gender,age,Education_Type,education_id,Employment_type,custLoanJourneyCity,custLoanJourneyProvince,custLoanJourneyZipCode,loanapplication_latitude,loanapplication_longitude,province,city,postalCode,onboardinggeolocation,incomeperSalaryScale,monthlyIncome,maritalStatus,purposeDescription,mer_category,isfatca,docType,loanRequestAmount,loanRequestTenure,monthlyInterestRate,interest_rate_offer,monthlyIncomeCheck,kycStatus,Device_os_type,obsFSPD30,FSPD30,Device_OSVersion,deviceType,Network_provider,loanmaster_brand,devicemanufacturer,devicemodel,brand,customer_never_transacted_flag,fspd30_mature_flag,blacklisted_customer_flag,writtenoff_flag,hackedordefrauded_flag,VasFlag,EverReinstallFlag,credolabScore,customerId_y,Appsflyer_id,Source,flag_organic_inorganic,cntevents
84116,b815ddad-ae3e-4685-a773-ff9a4b360a9d,1454336,60814543360023,2022-07-09 15:59:18,2024-08-24 10:01:25,2024-08-24 10:11:36,2024-08-24 10:09:34,2024-08-24 10:52:04,Quick,M,49,College Graduate,14,None,SAN MATEO,RIZAL,1850,14.677871,121.131143,Rizal,SAN MATEO,1850,14.677849 | 121.1311209,50233.00,40000,Married,Home Repair & Improvement,None,0,Philippines - Social Security Card,5000.0,6,7.916667,None,true,SKYC,android,0,0,14.000000000,Android,Globe/TM,samsung,None,SM-A042F,samsung,0,0,0,0,0,1,0,451.0,1454336,1705490550027-2131312284949265664,Organic,Organic,200
34468,88c7911c-a6cb-4c1e-8dba-4e719ab8a29d,2691873,60826918730012,2024-07-26 13:10:46,2024-07-26 13:14:00,2024-07-26 13:20:55,2024-07-26 13:20:44,2024-07-26 13:24:10,SIL-Instore,M,44,High School Graduate,11,None,ROSARIO,BATANGAS,4225,13.845235,121.205592,Batangas,ROSARIO,4225,13.8452323 | 121.2055635,35419.00,20000,Married,Mobile Phones,Mobile Retailer,0,Philippines - Postal Id Card (2016),7999.0,10,6.640000,None,true,SKYC,android,0,0,14.000000000,Android,Smart,OPPO,None,RMX3630,realme,0,0,0,0,0,1,0,417.0,<NA>,NaN,NaN,NaN,0
12791,d2283d88-c0c3-4d84-bb17-79916ae94b3f,2390047,60823900470014,2024-02-04 18:47:36,2024-02-04 18:49:16,2024-02-04 18:52:40,2024-02-04 18:52:32,2024-02-04 18:57:11,SIL-Instore,F,38,College Graduate,14,None,NOVELETA,CAVITE,4105,14.415432,120.855701,Cavite,NOVELETA,4105,14.4794543 | 120.8976549,28066.00,27000,Married,Televisions,Appliances Retailer,0,Philippines - Professional Id Card (2019),10319.0,6,4.500000,None,true,SKYC,android,1,0,14.000000000,Android,Globe/TM,vivo,None,V2317,vivo,0,1,0,0,0,1,0,500,<NA>,NaN,NaN,NaN,0
21862,27282e1b-5190-4308-a79e-4b070995e435,2835670,60828356700011,2024-09-10 15:50:18,2024-09-10 15:52:51,2024-09-10 16:07:11,2024-09-10 15:55:14,2024-09-10 16:17:47,SIL-Instore,M,37,High School Graduate,11,None,MALOLOS,BULACAN,3000,14.843517,120.811778,Bulacan,MALOLOS,3000,14.7979169 | 120.8718506,103620.00,30000,Married,Televisions,Appliances Retailer,0,Philippines - Driving License (2023),5395.0,6,3.340000,None,None,SKYC,android,0,0,14.000000000,Android,Globe/TM,vivo,None,V2317,vivo,0,0,0,0,0,1,0,454.0,<NA>,NaN,NaN,NaN,0
14361,111e824b-33fe-4d2f-8289-9d0c26bfb776,2561492,60825614920012,2024-06-08 16:58:25,2024-06-08 17:03:36,2024-06-08 17:07:34,2024-06-08 17:07:15,2024-06-08 17:13:32,SIL-Instore,M,39,College Graduate,14,None,SAN JOSE DEL MONTE,BULACAN,3023,14.801410,121.034254,Bulacan,SAN JOSE DEL MONTE,3023,null | null,40000,40000,Live-in Partner,Televisions,Appliances Retailer,0,Philippines - Driving License (2022),16989.0,12,5.080000,None,false,SKYC,android,1,0,12.000000000,Android,Globe/TM,samsung,None,SM-J610G,samsung,0,1,0,0,0,1,0,455,<NA>,NaN,NaN,NaN,0


In [25]:
finaldf.columns

Index(['digitalLoanAccountId', 'customerId', 'loanAccountNumber',
       'tsa_onboarding_time', 'startApplyDateTime', 'decision_date',
       'termsAndConditionsSubmitDateTime', 'disbursementDateTime',
       'new_loan_type', 'gender', 'age', 'Education_Type', 'education_id',
       'Employment_type', 'custLoanJourneyCity', 'custLoanJourneyProvince',
       'custLoanJourneyZipCode', 'loanapplication_latitude',
       'loanapplication_longitude', 'province', 'city', 'postalCode',
       'onboardinggeolocation', 'incomeperSalaryScale', 'monthlyIncome',
       'maritalStatus', 'purposeDescription', 'mer_category', 'isfatca',
       'docType', 'loanRequestAmount', 'loanRequestTenure',
       'monthlyInterestRate', 'interest_rate_offer', 'monthlyIncomeCheck',
       'kycStatus', 'Device_os_type', 'obsFSPD30', 'FSPD30',
       'Device_OSVersion', 'deviceType', 'Network_provider', 'brand',
       'devicemanufacturer', 'devicemodel', 'customer_never_transacted_flag',
       'fspd30_mature_flag

In [23]:
finaldf.rename(columns={'customerId_x':'customerId'}, inplace = True)


In [26]:
finaldf = finaldf[['digitalLoanAccountId', 'customerId', 'loanAccountNumber',
       'tsa_onboarding_time', 'startApplyDateTime', 'decision_date',
       'termsAndConditionsSubmitDateTime', 'disbursementDateTime',
       'new_loan_type', 'gender', 'age', 'Education_Type', 'education_id',
       'Employment_type', 'custLoanJourneyCity', 'custLoanJourneyProvince',
       'custLoanJourneyZipCode', 'loanapplication_latitude',
       'loanapplication_longitude', 'province', 'city', 'postalCode',
       'onboardinggeolocation', 'incomeperSalaryScale', 'monthlyIncome',
       'maritalStatus', 'purposeDescription', 'mer_category', 'isfatca',
       'docType', 'loanRequestAmount', 'loanRequestTenure',
       'monthlyInterestRate', 'interest_rate_offer', 'monthlyIncomeCheck',
       'kycStatus', 'Device_os_type', 'obsFSPD30', 'FSPD30',
       'Device_OSVersion', 'deviceType', 'Network_provider', 'brand',
       'devicemanufacturer', 'devicemodel', 'customer_never_transacted_flag',
       'fspd30_mature_flag', 'blacklisted_customer_flag', 'writtenoff_flag',
       'hackedordefrauded_flag', 'VasFlag', 'EverReinstallFlag',
       'credolabScore', 'Appsflyer_id', 'Source', 'flag_organic_inorganic',
       'cntevents']].copy()

In [27]:
finaldf.sample(5)

,digitalLoanAccountId,customerId,loanAccountNumber,tsa_onboarding_time,startApplyDateTime,decision_date,termsAndConditionsSubmitDateTime,disbursementDateTime,new_loan_type,gender,age,Education_Type,education_id,Employment_type,custLoanJourneyCity,custLoanJourneyProvince,custLoanJourneyZipCode,loanapplication_latitude,loanapplication_longitude,province,city,postalCode,onboardinggeolocation,incomeperSalaryScale,monthlyIncome,maritalStatus,purposeDescription,mer_category,isfatca,docType,loanRequestAmount,loanRequestTenure,monthlyInterestRate,interest_rate_offer,monthlyIncomeCheck,kycStatus,Device_os_type,obsFSPD30,FSPD30,Device_OSVersion,deviceType,Network_provider,brand,devicemanufacturer,devicemodel,customer_never_transacted_flag,fspd30_mature_flag,blacklisted_customer_flag,writtenoff_flag,hackedordefrauded_flag,VasFlag,EverReinstallFlag,credolabScore,Appsflyer_id,Source,flag_organic_inorganic,cntevents
33177,ee32580e-983a-463c-acc8-63a51f32d834,2319497,60823194970012,2023-11-30 17:49:42,2023-11-30 17:58:04,2023-11-30 18:06:06,2023-11-30 18:05:58,2023-11-30 18:18:08,SIL-Instore,F,55,College Undergraduate,13,Permanent Job (Private sector),CITY OF CALOOCAN,METRO MANILA,1400,14.658210,120.962092,METRO MANILA,CITY OF CALOOCAN,1400,null | null,19202.00,18000,Married,"Washing machine, dryer or both",Appliances Retailer,0,Philippines - Social Security Card #1,9549.00,9,4.500000,None,true,SKYC,android,1,0,8.000000000,Android,Globe/TM,realme,None,RMX3201,0,1,0,0,0,0,0,452,NaN,NaN,NaN,0
123183,da0baf79-3792-47cd-9d9f-7ef0a4d745a7,2644060,60826440600011,2024-07-08 14:54:10,2024-07-08 14:59:47,2024-07-08 15:09:51,2024-07-08 15:09:39,2024-07-08 15:15:55,SIL-Instore,F,48,High School Graduate,11,None,TANAUAN,BATANGAS,4232,14.083664,121.147440,Batangas,TANAUAN,4232,null | null,15000,15000,Married,"Washing machine, dryer or both,Small Appliances",Appliances Retailer,0,Philippines - Id Card (2020),5098.00,6,4.940000,None,None,SKYC,android,1,0,14.000000000,Android,TNT,OPPO,None,CPH2343,0,1,0,0,0,1,0,393.0,NaN,NaN,NaN,28
132601,af65120e-8ad0-451b-b98d-9a03da9bfc42,2795531,60827955310015,2024-08-26 18:56:59,2024-08-26 19:01:17,2024-08-26 19:16:19,2024-08-26 19:05:55,2024-08-26 19:48:17,Quick,F,31,College Graduate,14,None,BACOOR,CAVITE,4102,14.382561,121.006464,Cavite,BACOOR,4102,14.3825648 | 121.0064563,25170.00,69000,Single,Medical or Other Emergency/Health Care,None,0,Philippines - UMID Card,20000.00,9,7.916667,None,None,SKYC,android,0,0,14.000000000,Android,Sun,samsung,None,SM-F946B,0,0,0,0,0,0,0,468.0,1724669464644-5232853938194215122,Aff_Pokkt,InOrganic,0
92741,f2d59db3-298d-48b1-a73e-2a2e6aa6e52d,2820046,60828200460018,2024-09-03 16:23:21,2024-09-03 16:26:28,2024-09-03 16:34:06,2024-09-03 16:33:53,2024-09-03 16:38:16,SIL ZERO,M,53,College Graduate,14,Permanent Job (Private sector),LIPA CITY,BATANGAS,4217,13.941924,121.150585,Batangas,LIPA CITY,4217,13.939982599705113 | 121.14310962811935,26608.00,37000,Single,Mobile Phones,Mobile Retailer,0,Philippines - UMID Card (2016),7998.00,8,0.000000,None,true,SKYC,ios,0,0,18.000000000,iOS,Smart,None,None,None,0,0,0,0,0,1,0,None,NaN,NaN,NaN,0
107353,5d763d2a-6bc1-482c-86aa-6130e104cb8d,1373709,60813737090031,2022-05-14 07:24:29,2024-05-27 08:03:19,2024-05-27 08:10:31,NaT,2024-05-27 08:10:31,Flex-up,F,34,None,None,None,CITY OF CALOOCAN,METRO MANILA,1400,NaN,NaN,METRO MANILA,CITY OF CALOOCAN,1400,14.6547751 | 120.9705444,None,28000,Single,Living Expenses,None,0,Philippines - ePassport,31987.52,24,4.000000,48,None,SKYC,android,1,0,13.000000000,Android,Globe/TM,Redmi,None,M2101K6G,0,1,0,0,0,0,0,None,NaN,NaN,NaN,123


In [28]:
finaldf.rename(columns= {'Source':'AquisitionChannel'}, inplace = True)

In [29]:
finaldf.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\demographic_data_20241017.csv", index = False)

In [30]:
finaldf.dtypes

digitalLoanAccountId                        object
customerId                                   Int64
loanAccountNumber                           object
tsa_onboarding_time                 datetime64[us]
startApplyDateTime                  datetime64[us]
decision_date                       datetime64[us]
termsAndConditionsSubmitDateTime    datetime64[us]
disbursementDateTime                datetime64[us]
new_loan_type                               object
gender                                      object
age                                          Int64
Education_Type                              object
education_id                                object
Employment_type                             object
custLoanJourneyCity                         object
custLoanJourneyProvince                     object
custLoanJourneyZipCode                      object
loanapplication_latitude                   float64
loanapplication_longitude                  float64
province                       

In [31]:
finaldf.mer_category.unique()                               

array(['Appliances Retailer', None, 'Mobile Retailer',
       'Home Appliances Retailer', 'Home and Furnitures Retailer'],
      dtype=object)

In [32]:
finaldf.to_pickle(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demo&Telco Scorecard\Data\demographic_data_20241017.pkl")